In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['PYOPENGL_PLATFORM'] = 'egl'

from pathlib import Path
import trimesh
import pyrender
import h5py
import numpy as np
import torch
import os, os.path as osp
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.feature import plot_matches
import cv2
from PIL import Image

from datasets.scannet.utils_3d import ProjectionHelper, adjust_intrinsic, make_intrinsic, load_intrinsic, load_pose
from datasets.scannet.utils_3d import load_depth, load_color
from scripts.sem_seg.prep_backproj_data import get_world_to_scene

In [ ]:
def get_scan_name(scene_id, scan_id):
    return f'scene{str(scene_id).zfill(4)}_{str(scan_id).zfill(2)}'

# globals
subvol_size = (32, 32, 32)
voxel_size = 0.05
voxel_dims = (1, 1, 1)
root = Path('/mnt/data/scannet/scans')
proj_img_size = (40, 30)

data_dir = Path('/mnt/data/scannet/backproj')
fname = 'train100-v6.h5'
f = h5py.File(data_dir / fname, 'r')



In [ ]:
f['x'].shape

In [ ]:
for ndx in tqdm(range(f['x'].shape[0])):
    if f['x'][ndx].max() != 1:
        print(ndx)
        break

In [ ]:
occupied, projected, overlap = [], [], []

for ndx in tqdm(range(1000)):
    w2g, sceneid, scanid, frames = f['world_to_grid'][ndx], f['scene_id'][ndx], f['scan_id'][ndx], f['frames'][ndx]

    subvol_x = f['x'][ndx]
    # per-scene basics
    scan_name = get_scan_name(sceneid, scanid)
    frame_ndx = 0
    pose_path = root / scan_name / 'pose' / f'{frames[frame_ndx]}.txt'
    pose = load_pose(pose_path).numpy()
    depth_path = root / scan_name / 'depth' / f'{frames[frame_ndx]}.png' 
    depth = load_depth(depth_path, proj_img_size)
    rgb_path = root / scan_name / 'color' / f'{frames[frame_ndx]}.jpg' 
    rgb = load_color(rgb_path, proj_img_size)
    # get projection
    intrinsic_path = root / scan_name / 'intrinsic/intrinsic_color.txt'
    intrinsic = load_intrinsic(intrinsic_path)
    # adjust for smaller image size
    intrinsic = adjust_intrinsic(intrinsic, [1296, 968], proj_img_size)

    projection = ProjectionHelper(
                intrinsic, 
                0.4, 4.0,
                proj_img_size,
                subvol_size, voxel_size
            )

    # projection expects origin of chunk in a corner
    # but w2g is wrt center of the chunk -> add 16 to its "grid coords" to get the required grid coords
    # ie 0,0,0 becomes 16,16,16
    # add an additional translation to existing one 
    t = torch.eye(4)
    t[:3, -1] = 16
    w2g_tmp = t @ w2g

    proj = projection.compute_projection(torch.Tensor(depth), torch.Tensor(pose), torch.Tensor(w2g_tmp))
    if proj is None: 
        continue
    proj3d, proj2d = proj
    num_inds = proj3d[0]

    ind3d = proj3d[1:1+num_inds]
    ind2d = proj2d[1:1+num_inds]

    coords_3d = torch.empty(4, num_inds)
    coords_3d = ProjectionHelper.lin_ind_to_coords_static(ind3d, coords_3d, subvol_size).T[:, :-1].long()
    i,j,k = coords_3d.T

#     print('Num correspondences:', proj3d[0].item())
#     print('Occupied voxels:', (subvol_x == 1).sum())
#     print('Overlap: ', (subvol_x[i, j, k] == 1).sum())
    projected.append(proj3d[0].item())
    occupied.append((subvol_x == 1).sum())
    overlap.append((subvol_x[i, j, k] == 1).sum())



In [ ]:
projected = torch.Tensor(projected)
occupied = torch.Tensor(occupied)
overlap = torch.Tensor(overlap)

In [ ]:
def stats(x):
    print(x.min(), x.max(), x.mean())

In [ ]:
stats(projected)

In [ ]:
stats(occupied)

In [ ]:
stats(overlap)